In [51]:
import pandas as pd
import re
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import make_scorer, r2_score
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib


In [8]:
df=pd.read_csv('vichle_data.csv')

In [9]:
df.head(5)

,Model Name,Manufacturer,Registered In,Model Date,Fuel Type,Transmission,Color,Body Type,Engine Displacement,Driven,Price,page_link
0,Toyota Corolla GLi 1.3 VVTi 2018,Toyota,punjab,2018,Petrol,Manual,White,Sedan,1300 cc,"160,177 km",3850000,https://www.pakwheels.com/used-cars/toyota-cor...
1,Toyota Corolla Altis Grande CVT-i 1.8 2015,Toyota,punjab,2015,Petrol,Automatic,Super white,Sedan,1800 cc,"80,200 km",4700000,https://www.pakwheels.com/used-cars/toyota-cor...
2,Suzuki Cultus EURO II 2015,Suzuki,punjab,2015,Petrol,Manual,White,Hatchback,1000 cc,"80,000 km",1625000,https://www.pakwheels.com/used-cars/suzuki-cul...
3,Suzuki Wagon R VXL 2022,Suzuki,punjab,2022,Petrol,Manual,Solid White,Hatchback,1000 cc,"43,000 km",3000000,https://www.pakwheels.com/used-cars/suzuki-wag...
4,Toyota C-HR S 2018,Toyota,punjab,2018,Hybrid,Automatic,Black,Crossover,1800 cc,"111,673 km",8500000,https://www.pakwheels.com/used-cars/toyota-c-h...


In [10]:
df.count()

Model Name             11099
Manufacturer           11099
Registered In          11099
Model Date             11099
Fuel Type              11099
Transmission           11099
Color                  11099
Body Type               9913
Engine Displacement    10989
Driven                 11099
Price                  11099
page_link              11099
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Model Name           11099 non-null  object
 1   Manufacturer         11099 non-null  object
 2   Registered In        11099 non-null  object
 3   Model Date           11099 non-null  int64 
 4   Fuel Type            11099 non-null  object
 5   Transmission         11099 non-null  object
 6   Color                11099 non-null  object
 7   Body Type            9913 non-null   object
 8   Engine Displacement  10989 non-null  object
 9   Driven               11099 non-null  object
 10  Price                11099 non-null  int64 
 11  page_link            11099 non-null  object
dtypes: int64(2), object(10)
memory usage: 1.0+ MB


In [14]:
df['Model Name'].unique()
df['Model Name'].nunique()

3313

In [16]:
df['Manufacturer'].unique()

array(['Toyota', 'Suzuki', 'DFSK', 'Honda', 'Nissan', 'Mazda', 'KIA',
       'MG', 'Daihatsu', 'Mercedes Benz', 'BAIC', 'Proton', 'Haval',
       'Hyundai', 'Changan', 'Mitsubishi', 'Audi', 'Adam', 'FAW', 'BYD',
       'BMW', 'Jeep', 'Lexus', 'JAC', 'Prince', 'Peugeot', 'Volkswagen',
       'Chevrolet', 'Chery', 'Daewoo', 'Isuzu', 'Porsche', 'MINI',
       'Subaru', 'Land Rover', 'Range Rover', 'United', 'Fiat', 'Seres',
       'Ford', 'SsangYong', 'Datsun', 'Deepal', 'Ferrari', 'Tesla',
       'Sogo', 'JW Forland', 'Rinco'], dtype=object)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Model Name           11099 non-null  category
 1   Manufacturer         11099 non-null  category
 2   Registered In        11099 non-null  category
 3   Model Date           11099 non-null  int64   
 4   Fuel Type            11099 non-null  category
 5   Transmission         11099 non-null  category
 6   Color                11099 non-null  category
 7   Body Type            11099 non-null  category
 8   Engine Displacement  11099 non-null  int64   
 9   Driven               11099 non-null  int64   
 10  Price                11099 non-null  int64   
dtypes: category(7), int64(4)
memory usage: 507.7 KB


In [23]:
df['Body Type'] = df['Body Type'].fillna(df['Body Type'].mode()[0])

In [21]:
df['Engine Displacement'] = df['Engine Displacement'].fillna(df['Engine Displacement'].mode()[0])

In [18]:
df.drop('page_link',axis=1 ,inplace=True)

In [31]:
df.head(5)

,Model Name,Manufacturer,Registered In,Model Date,Fuel Type,Transmission,Color,Body Type,Engine Displacement,Driven,Price
0,Toyota Corolla GLi 1.3 VVTi,Toyota,punjab,2018,Petrol,Manual,White,Sedan,1300,160177,3850000
1,Toyota Corolla Altis Grande CVT-i 1.8,Toyota,punjab,2015,Petrol,Automatic,Super white,Sedan,1800,80200,4700000
2,Suzuki Cultus EURO II,Suzuki,punjab,2015,Petrol,Manual,White,Hatchback,1000,80000,1625000
3,Suzuki Wagon R VXL,Suzuki,punjab,2022,Petrol,Manual,Solid White,Hatchback,1000,43000,3000000
4,Toyota C-HR S,Toyota,punjab,2018,Hybrid,Automatic,Black,Crossover,1800,111673,8500000


In [25]:
df['Engine Displacement'] = df['Engine Displacement'].str.replace(' cc', '').astype(int)

In [27]:
df['Driven'] = df['Driven'].str.replace(',', '').str.replace(' km', '').astype(int)


remove the year at the end of model name 

In [30]:
pattern = r'\s?\b\d{4}\b$'
df['Model Name'] = df['Model Name'].str.replace(pattern, '', regex=True)

remove brand name of car company from start point

In [36]:
df['Model Name'] = df['Model Name'].str.replace(r'^\w+\s+', '', regex=True)

In [38]:
df.head(1)

,Model Name,Manufacturer,Registered In,Model Date,Fuel Type,Transmission,Color,Body Type,Engine Displacement,Driven,Price
0,Corolla GLi 1.3 VVTi,Toyota,punjab,2018,Petrol,Manual,White,Sedan,1300,160177,3850000


In [28]:
features = ['Model Name','Manufacturer', 'Registered In','Model Date','Fuel Type', 'Transmission',
            'Color', 'Body Type','Engine Displacement', 'Driven']
target = 'Price'
print(len(features))

10


Convert all categorical columns to category dtype

In [26]:

for col in ['Model Name','Manufacturer', 'Registered In','Fuel Type', 'Transmission',
            'Color', 'Body Type']:
    df[col] = df[col].astype('category')


In [63]:
df.to_csv('cleaned_file.csv', index=False)

In [13]:
df=pd.read_csv('cleaned_data.csv')

In [20]:
df['Manufacturer'].nunique()

48

In [14]:
df.head(1)

,Model Name,Manufacturer,Registered In,Model Date,Fuel Type,Transmission,Color,Body Type,Engine Displacement,Driven,Price
0,Corolla GLi 1.3 VVTi,Toyota,punjab,2018,Petrol,Manual,White,Sedan,1300,160177,3850000


In [29]:
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2)

In [13]:
model = lgb.LGBMRegressor()


In [14]:
model.fit(X_train, y_train, categorical_feature=['Model Name','Manufacturer', 'Registered In','Fuel Type', 'Transmission','Color', 'Body Type'])

d:\Carsadviser\venv\Lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


LGBMRegressor()

In [15]:
preds = model.predict(X_test)

In [17]:
r2 = r2_score(y_test, preds)
print(f'R² Score: {r2}')

R² Score: 0.8782853789389424


In [18]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Calculate R² score
r2 = r2_score(y_test, preds)
print(f'R² Score: {r2}')

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, preds)
print(f'Mean Absolute Error (MAE): {mae}')

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, preds)
print(f'Mean Squared Error (MSE): {mse}')

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')


R² Score: 0.8782853789389424
Mean Absolute Error (MAE): 782511.0975084578
Mean Squared Error (MSE): 6143044646197.02
Root Mean Squared Error (RMSE): 2478516.6221345016


In [21]:
pickle.dump(model, open('model.pkl', 'wb'))

In [4]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [25]:
df.head(2)

,Model Name,Manufacturer,Registered In,Model Date,Fuel Type,Transmission,Color,Body Type,Engine Displacement,Driven,Price
0,Corolla GLi 1.3 VVTi,Toyota,punjab,2018,Petrol,Manual,White,Sedan,1300,160177,3850000
1,Corolla Altis Grande CVT-i 1.8,Toyota,punjab,2015,Petrol,Automatic,Super white,Sedan,1800,80200,4700000


In [ ]:
import pandas as pd

# New data input as a pandas DataFrame (assuming you have similar features as in the model)
new_data = pd.DataFrame({
    'Model Name':['Swift DLX 1.3 navigation'],
    'Manufacturer':['Suzuki'],
    'Registered In':['Punjab'],
    'Model Date':[2019],
    'Fuel Type':['Petrol'],
    'Transmission':['Manual'],
    'Color':['White'],
    'Body Type':['Hatchback'],
    'Engine Displacement':[1300],
    'Driven':[79000]
})

for col in ['Model Name','Manufacturer', 'Registered In','Fuel Type', 'Transmission',
            'Color', 'Body Type']:
    new_data[col] = new_data[col].astype('category')


# Make predictions
predictions = model.predict(new_data)

print(f'Predicted Price: {predictions[0]}')


In [22]:
param_grid = {
    'num_leaves': [31, 50, 70, 100],
    'learning_rate': [0.05, 0.1, 0.15, 0.2],
    'n_estimators': [100, 200, 500],
    'max_depth': [-1, 10, 20, 30],
    'min_child_samples': [20, 30, 50],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}


In [23]:
lgb_model = lgb.LGBMRegressor()


In [24]:
random_search = RandomizedSearchCV(
    estimator=lgb_model,
    param_distributions=param_grid,
    n_iter=30,  # Number of combinations to try
    scoring=make_scorer(r2_score),  # You can use others like neg_mean_absolute_error
    cv=5,  # 5-fold cross-validation
    verbose=1,
    random_state=42,
    n_jobs=-1
)


In [30]:
random_search.fit(
    X_train,
    y_train,
    categorical_feature=['Model Name', 'Manufacturer', 'Registered In',
                         'Fuel Type', 'Transmission', 'Color', 'Body Type']
)


Fitting 5 folds for each of 30 candidates, totalling 150 fits


d:\Carsadviser\venv\Lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


RandomizedSearchCV(cv=5, estimator=LGBMRegressor(), n_iter=30, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'max_depth': [-1, 10, 20, 30],
                                        'min_child_samples': [20, 30, 50],
                                        'n_estimators': [100, 200, 500],
                                        'num_leaves': [31, 50, 70, 100],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42,
                   scoring=make_scorer(r2_score, response_method='predict'),
                   verbose=1)

In [31]:
print("Best Parameters:", random_search.best_params_)
print("Best R2 Score from CV:", random_search.best_score_)


Best Parameters: {'subsample': 0.8, 'num_leaves': 31, 'n_estimators': 200, 'min_child_samples': 20, 'max_depth': 20, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Best R2 Score from CV: 0.8380850178140682


In [33]:
best_model = random_search.best_estimator_

In [36]:
pickle.dump(best_model, open('model2.pkl', 'wb'))

In [37]:
with open('model2.pkl', 'rb') as f:
    model2 = pickle.load(f)

In [119]:
import pandas as pd

# New data input as a pandas DataFrame (assuming you have similar features as in the model)
# new_data = pd.DataFrame({
#     'Model Name':['Civic Oriel 1.8 i-VTEC CVT'],
#     'Manufacturer':['Toyota'],
#     'Registered In':['Punjab'],
#     'Model Date':[2017],
#     'Fuel Type':['Petrol'],
#     'Transmission':['Automatic'],
#     'Color':['White'],
#     'Body Type':['Hatchback'],
#     'Engine Displacement':[4600],
#     'Driven':[34270]
# })
new_data = pd.DataFrame({
    'Model Name': ['Civic Oriel 1.8 i-VTEC CVT'],
    'Manufacturer': ['Honda'],
    'Registered In': ['punjab'],
    'Model Date':[2021],
    'Fuel Type': ['Petrol'],
    'Transmission': ['Automatic'],
    'Color': ['Black'],
    'Body Type': ['Sedan'],
    'Engine Displacement':[1800],
    'Driven': [11300]  # Example: km
})


for col in ['Model Name','Manufacturer', 'Registered In','Fuel Type', 'Transmission',
            'Color', 'Body Type']:
    new_data[col] = new_data[col].astype('category')


# Make predictions
predictions = model2.predict(new_data)

print(f'Predicted Price: {predictions[0]}')


Predicted Price: 7952311.46020364


In [34]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

y_pred = best_model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


R² Score: 0.7543461720036609
MAE: 695650.0790626815
RMSE: 2814156.8125266843


In [73]:
df=pd.read_csv('cleaned_data.csv')

In [74]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = [
    'Model Name', 'Manufacturer', 'Registered In', 'Fuel Type',
    'Transmission', 'Color', 'Body Type'
]

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [75]:
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Model Name           11099 non-null  int64
 1   Manufacturer         11099 non-null  int64
 2   Registered In        11099 non-null  int64
 3   Model Date           11099 non-null  int64
 4   Fuel Type            11099 non-null  int64
 5   Transmission         11099 non-null  int64
 6   Color                11099 non-null  int64
 7   Body Type            11099 non-null  int64
 8   Engine Displacement  11099 non-null  int64
 9   Driven               11099 non-null  int64
 10  Price                11099 non-null  int64
dtypes: int64(11)
memory usage: 953.9 KB


In [77]:
df.to_csv('cleaned_data.csv', index=False)

In [78]:
X = df.drop(columns=['Price'])  
y = df['Price']                 

In [79]:
# from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [80]:
model3 = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [81]:



y_pred = model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))


R² Score: 0.8888773246657735
MAE: 475321.0102151437
RMSE: 1850882.8537070132


In [82]:
with open('model3.pkl', 'wb') as f:
    pickle.dump(model, f)

In [83]:
with open('model3.pkl', 'rb') as f:
    model_3 = pickle.load(f)

with open('label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

In [112]:
new_data = {
    'Model Name': 'Civic Oriel 1.8 i-VTEC CVT',
    'Manufacturer': 'Honda',
    'Registered In': 'punjab',
    'Model Date':2021,
    'Fuel Type': 'Petrol',
    'Transmission': 'Automatic',
    'Color': 'Black',
    'Body Type': 'Sedan',
    'Engine Displacement':1800,
    'Driven': 11300  # Example: km
}

# Safe transformation
for col in ['Model Name', 'Manufacturer', 'Registered In', 'Fuel Type', 
            'Transmission', 'Color', 'Body Type']:
    le = label_encoders[col]
    if new_data[col] in le.classes_:
        new_data[col] = le.transform([new_data[col]])[0]
    else:
        print(f"⚠️ '{new_data[col]}' not found in training data for column '{col}'. Using fallback -1.")
        new_data[col] = -1

In [89]:
print(label_encoders['Registered In'].classes_)

['KPK' 'azad-kashmir' 'balochistan' 'punjab' 'sindh']


In [113]:
new_data_df = pd.DataFrame([new_data])

# Predict the price using the trained model
predicted_price = model_3.predict(new_data_df)

print(f"The predicted price for the car is: {predicted_price[0]:,.2f}")

The predicted price for the car is: 9,448,199.99
